# Project 3: Where's Waldo?

Due Mar. 4th

So far, we've mostly focused on using imagery to do stuff for which it is better suited than a human: calculating camera locations from imagery, finding an optimal projective transform to stitch images together, and (soon) we'll be doing "structure from motion" in which we create 3D models of the world from collections of 2D images.  These are tasks primarily based around measuring things and doing calculations.  On the other side of the coin is object recognition (identifying the semantic content of a scene), and the best contemporary computer vision algorithms do object recognition at roughly the level of a 2 year old human (with some exceptions).  For this (mini-)project, we're going to delve into a topic that sort of straddles the line between these two general realms of computer vision.

As a motivating example, did you ever play the game Where's Waldo.  There are books filled with images like the following:
<img src='waldo_1.jpg'>
The objective, of course, is to find Waldo, the man in the red striped shirt and beanie wearing glasses.  He looks like this:
<img src='waldo_template.jpg'>
These scenes are (obviously) intended to have a bunch of visual clutter to make this task reasonably challenging.

Your task will be to come up with an algorithm that locates the template image (Waldo's face) and the target image (the larger scene).  This is called *template matching*, and it's a primitive form of feature recognition.  

## Implementation
### Template Matching
Template matching works in a way that is very similar to filtering:  slide the template image over every location in the target image, computing some sort of metric at each position.  In practice, one commonly used choice for an error metric is the one that you've already used for matching keypoint descriptors: z-normalized sum square error.  Another choice is [normalized cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Normalization).  Once these metrics have been computed, simply find the argmin (for SSE) or argmax (for NCC), and this will be the location of the best match.  

**Your task is to implement template matching.  Use 'waldo_template.jpg' as the template and 'waldo_1.jpg' as the target image.  Where's Waldo? **

### Not so fast!!!  What about scale!
Oh, no.  As it turns out, the template I've provided is not the same scale as the Waldo in the image.  To deal with this, you'll need to create an image pyramid for the template (See Szeliski 3.5, and [Mubarak Shah's lecture on this topic](https://www.youtube.com/watch?v=KO7jJt0WHag&feature=youtu.be)).  This essentially just means creating a sequence of downsampled images of the template, and trying each one in hopes that one of the resulting down-scaled templates matches the feature in the target image.  **Create a sequence of templates with which to perform feature matching, each one 1/2 the resolution of the previous (so 1/4 the total number of pixels).  To avoid aliasing, before downsampling perform a $\sigma=1$ Gaussian Blur of the image.  Once you've built your image pyramid, find the argmin/max in 3 dimensions (u,v,template scale)**.

## Generalization
**Waldo appears in every Where's Waldo image (obviously).  Try using the same technique on 'waldo_2.jpg'.  Does the algorithm work?**  I confess that I pulled the image of waldo for the template directly from 'waldo_1.jpg', so for the correct scale, there is something close to an exact match (i.e. SSE=0).  However, Waldo, while easily recognizable to the human eye after undergoing the small scale deformations associated with artistic license, is not so easily recognizable via template matching.  We will return to a similar problem when discussing object recognition, and hopefully this example will motivate the need to come up with representations of objects (like Waldo) that are more robust.


In [68]:
def gauss_kernal(size, var):
    kernel = np.zeros(shape=(size,size))
    for i in range(size):
        for j in range(size):
            kernel[i][j] = mt.exp( -((i - (size-1)/2)**2 + (j - (size-1)/2)**2 )/(2*var*var))
    kernel = kernel / kernel.sum()
    return kernel

def convolve(g,template): # h is kernel, g is the image
    I_gray_copy = g.copy()
    h,w = template.shape
    minSSE = np.inf
    bestX = 0
    bestY = 0
    wl = int(w/2)
    hl = int(h/2)
    for i in range(hl,len(g[0])-hl-1):
        for j in range(wl, len(g)-wl-1):
            patch = g[i-hl:i+hl+1,j-wl:j+wl+1]
            sse = sum_squared_error(template,patch)
            sse = sum(sum(sse))       
            if(sse < minSSE):
                minSSE = sse
                bestX = i
                bestY = j
        if(i%10 == 0):
            print(i,minSSE,bestX,bestY)
    return minSSE,i,j

def resize(image):
    size =image.size
    ratio = 0.5  
    reduced_size = int(size[0] * ratio), int(size[1] * ratio)
    image = image.resize(reduced_size, Image.ANTIALIAS)
    return image

def sum_squared_error(D1, D2):
    if(D1.shape == D2.shape):
        return (((D1-np.mean(D1))/np.std(D1)) - ((D2-np.mean(D2))/np.std(D2)))**2
    else:
        return np.inf
    


In [69]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


I1 = plt.imread('waldo_1.jpg')
I1 = I1.mean(axis=2)
template = [0,0,0,0]

template[0] = Image.open('waldo_template.jpg').convert('L')
template[1] = resize(template[0])
template[2] = resize(template[1])
template[3] = resize(template[2])
template = np.asarray(template[2])


print(convolve(I1,template))

[[ 70  64  75  92 140 180 205 220 216 196 149  87  41  36  52  52  45  45
   37  46 124 206 220 220 185]
 [ 64  67 154 214 247 255 245 209 163 103  59  38  39  58  78  70  68  80
   71  43  71 160 227 236 154]
 [ 83 103 221 254 250 235 164  66  52  49  62  79  86  88  89  79  73  87
   90  55  52 130 222 220 116]
 [ 74 138 252 227 186 137  68  53  64  85 102  99  83  71  69  59  43  49
   58  41  34  76 139 136  69]
 [ 74  95 174 125  74  56  42  86  92  97  93  71  46  36  41  50  39  36
   42  36  28  36  50  45  36]
 [ 98  49  78  50  42  76  87  83  74  67  62  52  39  36  43  46  44  47
   47  42  38  34  27  24  35]
 [107  48  75  80  79  85  93  85  62  45  42  42  42  46  47  40  39  41
   39  37  36  37  37  42  42]
 [ 65  43  80 100  95  82  76  55  46  39  38  43  39  34  41  37  39  42
   35  28  30  38  47  39  35]
 [ 41  57  81  86  80  74  68  37  34  39  26  31  31  17  36  38  36  40
   39  33  31  34  33  24  28]
 [ 64  48  57  74  56  35  37  37  49  69  57  45  58  

/home/mathew/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


160 781.7736801550188 146 726
170 781.7736801550188 146 726
180 781.7736801550188 146 726
190 781.7736801550188 146 726
200 781.7736801550188 146 726
210 781.7736801550188 146 726
220 781.7736801550188 146 726
230 781.7736801550188 146 726
240 781.7736801550188 146 726
250 781.7736801550188 146 726
260 781.7736801550188 146 726
270 781.7736801550188 146 726
280 781.7736801550188 146 726
290 781.7736801550188 146 726
300 781.7736801550188 146 726
310 781.7736801550188 146 726
320 781.7736801550188 146 726
330 781.7736801550188 146 726
340 781.7736801550188 146 726
350 781.7736801550188 146 726
360 768.9258484152602 359 543
370 768.9258484152602 359 543
380 768.9258484152602 359 543
390 768.9258484152602 359 543
400 768.9258484152602 359 543
410 768.9258484152602 359 543
420 768.9258484152602 359 543
430 768.9258484152602 359 543
440 768.9258484152602 359 543
450 768.9258484152602 359 543
460 768.9258484152602 359 543
470 768.9258484152602 359 543
480 768.9258484152602 359 543
490 768.92

KeyboardInterrupt: 